In diesem notebook wurde gemessen, in welchen Dialogen Uneinigkeit herrscht. Die Posts aus diesen Dialogen wurden mit 1 gelabelt, die ohne Uneinigkeit mit 0.


In [ ]:
import pandas as pd

In [ ]:
data_full = pd.read_excel('data_full_clean.xlsx')
data_full


In [ ]:
def disagreement(data):
    data['disagreement'] = None
    data_on_topic = data[data['var_opinion']>-9]
    data_on_topic = data_on_topic[data_on_topic['var_opinion']!=0]
    grouped_df = data_on_topic.groupby(['4: tree_id', '11: path'])
    for index, group in grouped_df:
        authors = group.groupby(['13: author_flow'])
        path = group['11: path'].tolist()[0]
        id = group['4: tree_id'].tolist()[0]
        if authors.ngroups > 2:
            group = group[group['6: parent_id'].notna()]
            authors = group.groupby(['13: author_flow'])
        op_values = {}
        for i, author in authors:
            op_value = sum(author['var_opinion'].tolist())/len(author)
            op_values[i[0]] = op_value
        if len(op_values)==2:
            val_a = list(op_values.values())[0]
            val_b = list(op_values.values())[1]
            if (val_a<0 and val_b<0) or (val_a>0 and val_b>0):
                disagree = 0
            if (val_a<0 and val_b>0) or (val_a>0 and val_b<0):
                disagree = 1 
        else:
            continue
        data.loc[(data['4: tree_id'] == id) & (data['11: path'] == path), 'disagreement'] = disagree
    return data
    
        


dis_df = disagreement(data_full)

In [ ]:
dis_df.head(20)

In [ ]:
df = data_full[data_full['4: tree_id']==1652564083688848896]
df = df[df['11: path']==7]
df


In [ ]:
dis_df.to_excel('data_disagreement.xlsx')